# 라이브러리

In [1]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)



다음의 패키지를 부착합니다: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



다음의 패키지를 부착합니다: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


다음의 패키지를 부착합니다: ‘randomForest’


The following object is masked from ‘package:gridExtra’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



다음의 패키지를 부착합니다: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


필요한 패키지를 로딩중입니다: lattice



# 전처리

In [2]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [3]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [4]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [5]:
str(data)

'data.frame':	54819 obs. of  11 variables:
 $ 취업일자         : chr  "2020-01-01" "2020-01-01" "2020-01-01" "2020-01-01" ...
 $ 연령             : int  65 61 53 64 53 60 63 40 56 62 ...
 $ 장애유형         : Factor w/ 5 levels "시각장애","정신장애",..: 1 1 1 1 1 1 1 1 4 1 ...
 $ 중증여부         : Factor w/ 2 levels "경증","중증": 2 2 1 2 1 2 2 2 2 2 ...
 $ 근무지역         : Factor w/ 273 levels "강원 강릉시",..: 98 100 116 107 102 112 109 121 18 17 ...
 $ 취업직종대분류   : Factor w/ 23 levels "건설·채굴직",..: 2 2 2 2 2 2 2 9 9 9 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 5 5 4 5 4 5 5 3 4 5 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 2 1 2 1 2 2 2 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 1 1 1 1 1 5 6 10 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 4 4 4 4 4 4 4 1 1 ...
 $ target           : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...


In [6]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [7]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [8]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [9]:
# 근무지역 top 6 남기기 서울 경기 경남 부산 인천 경북 
data_filtered <- data_filtered %>%
  filter(근무지역분류 %in% c("서울"))

In [10]:
#   근무지역 분류 바꾸기

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"


In [11]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12

In [12]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	11996 obs. of  35 variables:
 $ 취업직종대분류   : Factor w/ 23 levels "건설·채굴직",..: 2 2 2 12 12 12 12 12 12 12 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 2 1 2 1 2 1 2 1 1 1 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 8 6 4 10 8 10 8 10 10 10 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 2 2 2 4 4 4 4 4 4 4 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ target           : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "서비스직" ...
 $ age_20           : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30           : logi  TRUE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증    : logi  FA

[1] 0

In [13]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	11996 obs. of  35 variables:
 $ 취업직종대분류   : Factor w/ 23 levels "건설·채굴직",..: 2 2 2 12 12 12 12 12 12 12 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 2 1 2 1 2 1 2 1 1 1 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 8 6 4 10 8 10 8 10 10 10 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 2 2 2 4 4 4 4 4 4 4 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ target           : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "서비스직" ...
 $ age_20           : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30           : logi  TRUE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증    : logi  FA

[1] 0

In [14]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [15]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	11996 obs. of  31 variables:
 $ 취업직종대분류: Factor w/ 23 levels "건설·채굴직",..: 2 2 2 12 12 12 12 12 12 12 ...
 $ target        : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "서비스직" ...
 $ age_20        : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30        : logi  TRUE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  TRUE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : log

[1] 0

In [16]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [17]:
str(data_filtered_subset)

'data.frame':	11996 obs. of  31 variables:
 $ 취업직종대분류: Factor w/ 23 levels "건설·채굴직",..: 2 2 2 12 12 12 12 12 12 12 ...
 $ target        : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "서비스직" ...
 $ age_20        : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30        : logi  TRUE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  TRUE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증 : log

# 변동사항

In [18]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [19]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [20]:
str(data_filtered_subset)

'data.frame':	11996 obs. of  30 variables:
 $ target       : Factor w/ 3 levels "경영·행정·사무직",..: 1 1 1 3 3 3 3 3 3 3 ...
 $ age_20       : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30       : logi  TRUE FALSE TRUE FALSE TRUE FALSE ...
 $ age_40       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  TRUE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE TRUE FALSE TRUE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FALSE F

In [21]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
          2.87153           -0.97378           -0.96092           -0.82225  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
         -0.55635           -0.21081                 NA           -0.22403  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
         -0.33522           -0.40219           -0.40503            0.01258  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
          0.44708           -0.18510           -0.13630           -0.60537  
정신장애_중증TRUE           서울TRUE            JanTRUE            FebTRUE  
               NA                 NA           -0.27357           -0.05501  
          MarTRUE            AprTRUE            MayTRUE            JunTRUE  
          0.03953           -0.03130            0.04954           -0.11239  
          JulTRUE            AugTRUE            SepTRUE       

# 시험준비

In [22]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [23]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [24]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [25]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=500)

In [ ]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [ ]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [ ]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =20)

# weights:  663
initial  value 7849.835449 
iter  10 value 3368.257074
iter  20 value 3179.633646
iter  30 value 3096.865861
iter  40 value 3013.527124
iter  50 value 2961.718803
iter  60 value 2915.786752
iter  70 value 2882.026427
iter  80 value 2850.044166
iter  90 value 2827.450199
iter 100 value 2809.193076
final  value 2809.193076 
stopped after 100 iterations


# 검증하기

In [ ]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [ ]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [ ]:
# KNN 검증하기
mean(knnmodel==y) 

[1] 0.586875

In [ ]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [ ]:
#  RF 예측력
mean(predrf == y)

[1] 0.6125

In [ ]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.598125

In [ ]:
#  KNN 예측력
mean(knnmodel == y)

[1] 0.586875

In [ ]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.585625

# 오분류표

In [ ]:
#  Random Forest 오분류표
table(predrf,y)

                  y
predrf             경영·행정·사무직 기타 서비스직
  경영·행정·사무직              454   93      191
  기타                           13   87       13
  서비스직                      214   96      439

In [ ]:
#  SVM 오분류표
table(predmodelsvm,y)

                  y
predmodelsvm       경영·행정·사무직 기타 서비스직
  경영·행정·사무직              444   97      203
  기타                           22   93       20
  서비스직                      215   86      420

In [ ]:
#  KNN 오분류표
# table(y, knnmodel)

In [ ]:
# nnet 오분류표
table(prednnet,y)

                  y
prednnet           경영·행정·사무직 기타 서비스직
  경영·행정·사무직              429   81      188
  기타                           29   94       41
  서비스직                      223  101      414

In [ ]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,경영·행정·사무직,기타,서비스직
1,0.062,0.918,0.02
